<a href="https://colab.research.google.com/github/shubhamgoel90/Pytorch-Basics/blob/master/rnn_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch 
from torch import nn 
from torch import optim
from torch.nn import functional as F

import numpy as np
import pandas as pd

In [0]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [0]:
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
text=open('/content/drive/My Drive/shakespeare.txt').read()[7800:].lower()

In [0]:
print (text[:100])

creatures we desire increase,
  that thereby beauty's rose might never die,
  but as the riper shoul


In [0]:
itos = list(set(text))
itos.insert(0,'unk')
itos.insert(0,'pad')
stoi = { x:i for i,x in enumerate(itos)}

In [0]:
itos[48]

'a'

In [0]:
data = [stoi[i] for i in text][:25000]

In [0]:
data[:5],[ itos[i] for i in data[:5]]

([2, 38, 23, 48, 32], ['c', 'r', 'e', 'a', 't'])

In [0]:
cs = 10
xs=np.array([data[ix:cs+ix] for ix in range(0,len(data)-cs)])

In [0]:
x_train = xs[:20000,:9]
y_train = xs[:20000,9]

x_val = xs[20000:,:9]
y_val = xs[20000:,9]

In [0]:
len(x_train),len(y_train),len(x_val),len(y_val)

(20000, 20000, 4990, 4990)

In [0]:
x_train[0],y_train[0]

(array([ 2, 38, 23, 48, 32, 54, 38, 23, 26]), 16)

In [0]:
''.join([itos[i] for i in x_train[0]]),itos[y_train[0]]

('creatures', ' ')

In [0]:
x_train, y_train, x_val, y_val = map(
    torch.tensor, (x_train, y_train, x_val, y_val)
)

In [0]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
bs = 512
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs)


val_ds = TensorDataset(x_val, y_val)
val_dl = DataLoader(val_ds, batch_size=bs)

In [0]:
# bs,seq_len --> bs,seq_len,emb --> bs,seq_len,hid --> LSTM --> bs,seq_len,vocab

In [0]:
class lstm_model(nn.Module):
  def __init__(self,vs,emb,hid):
    super().__init__()
    self.vs,self.emb,self.hid=vs, emb,hid
    self.emb = nn.Embedding(vs,emb)
    self.layer = nn.Linear(emb,hid)
    self.rnn = nn.RNN(hid,hid)
    self.out = nn.Linear(hid,vs)
  def forward(self,xb):
    e  = self.emb(xb)
    o = F.tanh(self.layer(xb))
    bs = len(xb)
    hid = torch.zeros(bs,self.hid)
    out,hid  = self.rnn(o,hid)
    print (out.shape)
    return F.log_softmax(self.out(out[:,-1]))


In [0]:
def mse(xb,xp):
  return ((xb-xp)**2).mean()

In [0]:
m = lstm_model(len(itos),100,128);m

lstm_model(
  (emb): Embedding(66, 100)
  (layer): Linear(in_features=100, out_features=128, bias=True)
  (rnn): RNN(128, 128)
  (out): Linear(in_features=128, out_features=66, bias=True)
)

In [0]:
class model(nn.Module):
  def __init__(self,vs,emb,hid):
    super().__init__()
    self.vs,self.emb,self.hid = vs,emb,hid
    self.emb =  nn.Embedding(vs,emb)
    self.l1= nn.Linear(emb,hid)
    self.rnn = nn.RNN(hid,hid,batch_first=True)
    self.out = nn.Linear(hid,vs)
  
  def forward(self,xb):
    bs = xb.shape[0]
    hid = torch.zeros((bs,self.hid),requires_grad=True)
    inp = self.emb(xb)
    inp = F.sigmoid(self.l1(inp))
    out,hid = self.rnn(inp) 
    print (out.shape)
    return F.log_softmax(self.out(out[:,-1]),dim=1)


In [0]:
m = model(len(itos),100,128);m

model(
  (emb): Embedding(66, 100)
  (l1): Linear(in_features=100, out_features=128, bias=True)
  (rnn): RNN(128, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=66, bias=True)
)

In [0]:
opt = optim.Adam(m.parameters(),lr=1e-3)

In [0]:
fit(20,m,F.cross_entropy,opt,train_dl,val_dl)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


0 2.855373767191518
1 2.645996330066291
2 2.4215269948772056
3 2.2787323020025343
4 2.1825726437425326
5 2.1174642035383022
6 2.0704538488674737
7 2.0319062318973886
8 1.9997156408841241
9 1.9718753654636696
10 1.9484026430126182
11 1.9265503534573114
12 1.9073958137947955
13 1.8907849270260644
14 1.8765236440306914
15 1.8643039958032672
16 1.8529305908149611
17 1.8415468961776855
18 1.8298121384484973
19 1.8194092883853492


In [0]:
(m(torch.tensor(data[:10])[None]).exp().detach().numpy())

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


array([[1.31160177e-05, 2.84023135e-05, 1.53070260e-02, 7.00378418e-02,
        9.52766204e-05, 1.99349288e-05, 6.89589158e-02, 8.44296446e-05,
        4.39821852e-05, 2.90087737e-05, 1.66967802e-05, 2.00599106e-03,
        3.94045562e-02, 3.92964408e-02, 4.83288983e-04, 2.82881047e-05,
        1.15406001e-02, 4.67356443e-02, 2.44178209e-05, 2.17226516e-05,
        5.33140302e-02, 2.31478605e-02, 2.58185640e-02, 1.47205172e-02,
        4.31337976e-05, 3.85300518e-05, 5.42619862e-02, 1.66297032e-05,
        4.10719404e-05, 4.92974686e-05, 2.06538243e-05, 2.76556420e-05,
        1.63160011e-01, 2.22341763e-03, 4.66442252e-05, 1.99374954e-05,
        6.80993107e-05, 1.92432217e-05, 1.69671513e-02, 8.39310233e-05,
        1.50210772e-05, 2.14451939e-04, 1.70759755e-04, 2.28000172e-05,
        6.86738268e-02, 2.77536809e-02, 6.53991592e-05, 6.77206144e-02,
        9.22604576e-02, 8.80084190e-06, 1.42693143e-05, 2.83460031e-05,
        2.71932259e-02, 2.26023680e-04, 4.21692384e-03, 1.081686

In [0]:
def generate(sent):
  inp = torch.tensor([stoi[i] for i in sent])
  p = m(inp[None]).exp().detach().numpy()[0]
  #print (sum(p))
  #return itos[np.argmax(p)]
  return (np.random.choice(itos,p=p))

In [0]:
def gen_n(sent,n=10):
  for i in range(n):
    sent+=generate(sent[-9:])
  return sent

In [0]:
print (gen_n('this is',n=1000))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


this is that my ig tell ginougrith thith codes my pake for my entimy for actees my het linde's nen ightil's thount of the sare, hourtron: and thee mave delte,
  anp porp theoh llaly sech stores from ay sombest nighte
  thy self babe digh with.
  shast ow owt ig an ryed pustom live wior thoo erssang theautet
  sorments your thou ard blions howning has it, eyed seet, simthen beas brow ceirtlm's then then sone gorpsteds eyat youngr in with the way thill when lovangine,
  a gite pithe,
  wyout'st by gren,
  the beret of geouty's shy a coongterend,
  i dath neade ightss shate,
  try meake anl'kid minger ftire, shoued youn youn the shounth cyet whing un thif mi dades is dath matast nich wall,
  what thine to pited,
    apken wirl,
  lovings a wite dith sim and the maken bespeds dreps.
  to their paavt

  for five,
  and oy partt dwate thouters me tome o'enn menuidst troust sy livt at;
  fo fot make old mave br doke propity she heo ant
  pur hach elight on thanmen minge teet het my wiesle on 

In [0]:
itos = ['the','is','"']